In [4]:
!pip install transformers 
!pip install datasets 


In [2]:
!pip install evaluate
!pip install peft


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 12.2 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset, DatasetDict, Dataset
from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
)
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np

In [7]:
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate

In [4]:
model_checkpoint = "distilbert-base-uncased"

# Define label maps
id2label = {0: "LEFT", 1: "RIGHT"}
label2id = {"LEFT": 0, "RIGHT": 1}

model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=2, id2label=id2label, label2id=label2id
)

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from datasets import load_dataset, concatenate_datasets

from transformers import BertTokenizerFast

from torch.utils.data import DataLoader

import pandas as pd


df1 = load_dataset(
    "csv", data_files="/kaggle/input/2017-1/2017_1.csv", encoding="utf-8"
)["train"]
df2 = load_dataset(
    "csv", data_files="/kaggle/input/2017-2/2017_2.csv", encoding="utf-8"
)["train"]
df = concatenate_datasets([df1, df2])

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [10]:
filtered_df = df.filter(lambda x: x["political_leaning"] in ["LEFT", "RIGHT"])

Filter:   0%|          | 0/326240 [00:00<?, ? examples/s]

In [11]:
filtered_df

Dataset({
    features: ['id', 'date_publish', 'outlet', 'headline', 'lead', 'body', 'authors', 'domain', 'url', 'political_leaning'],
    num_rows: 154401
})

In [12]:
filtered_df = filtered_df.train_test_split(test_size=0.1)

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
def tokenize_function(examples):
    text = examples["body"]
    labels = examples["political_leaning"]

    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text, return_tensors="np", padding=True, truncation=True, max_length=512
    )

    tokenized_inputs["labels"] = [label2id[label] for label in labels]
    return tokenized_inputs

In [9]:
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({"pad_token": "[PAD]"})
    model.resize_token_embeddings(len(tokenizer))

In [17]:
tokenized_dataset = filtered_df.map(tokenize_function, batched=True)
tokenized_dataset

Map:   0%|          | 0/138960 [00:00<?, ? examples/s]

Map:   0%|          | 0/15441 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'date_publish', 'outlet', 'headline', 'lead', 'body', 'authors', 'domain', 'url', 'political_leaning', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 138960
    })
    test: Dataset({
        features: ['id', 'date_publish', 'outlet', 'headline', 'lead', 'body', 'authors', 'domain', 'url', 'political_leaning', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 15441
    })
})

In [18]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
f1_metric = evaluate.load("f1")
recall = evaluate.load("recall")


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    return {
        "accuracy": accuracy.compute(predictions=predictions, references=labels),
        "precision": precision.compute(predictions=predictions, references=labels),
        "f1_score": f1_metric.compute(predictions=predictions, references=labels),
        "recall": recall.compute(predictions=predictions, references=labels),
    }

In [ ]:
text_list = [
    "It was good.",
    "Not a fan, don't recommended",
    "Better than the first one.",
    "Women have the right to choose and abortion should be allowed.",
]

import torch

# Set device
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model = model.to(device)

print("Untrained model")
for text in text_list:
    inputs = tokenizer(text, return_tensors="pt").to(
        device
    )  # Move inputs to the correct device
    logits = model(**inputs).logits  # Forward pass
    predictions = torch.argmax(logits, dim=-1)
    print(predictions.item())
    print(f"{text} - {id2label[predictions.item()]}")

Untrained model
0
It was good. - LEFT
0
Not a fan, don't recommended - LEFT
0
Better than the first one. - LEFT
0
Women have the right to choose and abortion should be allowed. - LEFT


In [22]:
peft_config = LoraConfig(
    task_type="SEQ_CLS", r=4, lora_alpha=32, lora_dropout=0.01, target_modules=["q_lin"]
)

In [23]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 628,994 || all params: 67,584,004 || trainable%: 0.9307


In [24]:
lr = 1e-3
batch_size = 10
num_epochs = 5

training_args = TrainingArguments(
    output_dir="" + model_checkpoint + "lora-txt",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [25]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [26]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy,Precision,F1 Score,Recall
1,0.324600,0.291280,{'accuracy': 0.8925587721002526},{'precision': 0.858729775416566},{'f1': 0.810853950518755},{'recall': 0.7680345572354211}
2,0.277900,0.289952,{'accuracy': 0.8968978693089826},{'precision': 0.8808926780341023},{'f1': 0.8152703643536784},{'recall': 0.7587473002159827}
3,0.277700,0.272824,{'accuracy': 0.9053817757917233},{'precision': 0.8597048808172532},{'f1': 0.8382955174322081},{'recall': 0.81792656587473}
4,0.259000,0.254758,{'accuracy': 0.9138009196295577},{'precision': 0.8736126840317101},{'f1': 0.8528468767274738},{'recall': 0.8330453563714902}
5,0.232400,0.231367,{'accuracy': 0.9212486237937957},{'precision': 0.8929558011049724},{'f1': 0.8644974370403389},{'recall': 0.8377969762419006}


Trainer is attempting to log a value of "{'accuracy': 0.8925587721002526}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.858729775416566}" of type <class 'dict'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.810853950518755}" of type <class 'dict'> for key "eval/f1_score" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'recall': 0.7680345572354211}" of type <class 'dict'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.8968978693089826}" of type <cl

TrainOutput(global_step=69480, training_loss=0.28088712247542147, metrics={'train_runtime': 15561.6153, 'train_samples_per_second': 44.648, 'train_steps_per_second': 4.465, 'total_flos': 9.33808894820352e+16, 'train_loss': 0.28088712247542147, 'epoch': 5.0})

In [27]:
trainer.evaluate()

Trainer is attempting to log a value of "{'accuracy': 0.9212486237937957}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.8929558011049724}" of type <class 'dict'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.8644974370403389}" of type <class 'dict'> for key "eval/f1_score" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'recall': 0.8377969762419006}" of type <class 'dict'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.23136654496192932,
 'eval_accuracy': {'accuracy': 0.9212486237937957},
 'eval_precision': {'precision': 0.8929558011049724},
 'eval_f1_score': {'f1': 0.8644974370403389},
 'eval_recall': {'recall': 0.8377969762419006},
 'eval_runtime': 152.657,
 'eval_samples_per_second': 101.148,
 'eval_steps_per_second': 10.121,
 'epoch': 5.0}

In [33]:
predictions = trainer.predict(tokenized_dataset["test"])

In [34]:
predictions

PredictionOutput(predictions=array([[ 4.842524 , -4.672748 ],
       [ 1.8642378, -1.9348584],
       [ 2.4659033, -2.3985987],
       ...,
       [ 2.5119832, -2.4510465],
       [ 7.287594 , -6.6343637],
       [ 1.6880037, -1.7174914]], dtype=float32), label_ids=array([0, 0, 0, ..., 0, 0, 0]), metrics={'test_loss': 0.23136654496192932, 'test_accuracy': {'accuracy': 0.9212486237937957}, 'test_precision': {'precision': 0.8929558011049724}, 'test_f1_score': {'f1': 0.8644974370403389}, 'test_recall': {'recall': 0.8377969762419006}, 'test_runtime': 152.3539, 'test_samples_per_second': 101.35, 'test_steps_per_second': 10.141})

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = model.to(device)
print("Trained model predictions")

for text in text_list:
    inputs = tokenizer.encode(text, return_tensors="pt").to(device)

    logits = model(inputs).logits

    predictions = torch.argmax(logits, dim=-1)
    print(f"{text} - {id2label[predictions.item()]}")

Using device: cuda
Trained model predictions
It was good. - LEFT
Not a fan, don't recommended - LEFT
Better than the first one. - LEFT
Women have the right to choose and abortion should be allowed. - LEFT


In [36]:
output_model_file = "/kaggle/working/pytorch_distilbert_imbd.bin"
output_vocab_file = "/kaggle/working/vocab_distilbert_imbd.bin"

# Save model
model_to_save = model
torch.save(model_to_save, output_model_file)

# Save tokenizer vocabulary in the current directory
tokenizer.save_vocabulary(".")  # Current directory

# Save model state dictionary
torch.save(model.state_dict(), "/kaggle/working/trained_model_gral_imbd_HEAD.pth")

print("All files saved")

All files saved
